# Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


# Installs

In [ ]:
!pip install --quiet geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 86.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 112.1 MB/s eta 0:00:00


In [ ]:
!wget https://github.com/9r4vEy4r6/taim-host/releases/download/aqi_tag/aqi_data.zip --quiet
!wget https://github.com/9r4vEy4r6/host/raw/main/telangana_ac.geojson --quiet
!unzip -q aqi_data.zip -d aqi/

# Imports

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Path, PathPatch
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from shapely.geometry import Point
import json

# Load Data

In [ ]:
data = gpd.read_file("telangana_ac.geojson")
data.to_crs(crs=3857) 
data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
aqi = pd.read_csv("aqi/merged.csv")

In [ ]:
dates = aqi.columns[1:-2]
dates

Index(['2017-01', '2017-02', '2017-03', '2017-04', '2017-05', '2017-06',
       '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12',
       '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06',
       '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
       '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
       '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
       '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
       '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
       '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', '2021-06',
       '2021-07', '2021-08', '2021-09', '2021-10', '2021-11', '2021-12'],
      dtype='object')

In [ ]:
!mkdir dataset
!mkdir dataset/images

In [ ]:
max = aqi.drop(["Location", "latitude", "longitude"], axis=1).max().max()
min = aqi.drop(["Location", "latitude", "longitude"], axis=1).min().min()
print(min, max)

24.0 153.0


In [ ]:
np.random.seed(1234)

def func(data_map, row) :
  aqis = []
  for i in data_map : 
    if row["geometry"].contains(i) :
      aqis.append(data_map[i])
  if len(aqis) == 0 : 
    return float('nan')
  else :
    return sum(aqis)/len(aqis)

for dt in tqdm(dates) : 
  day_datas = aqi[[dt, "latitude", "longitude"]]
  lats, lons, zs = [], [], []
  for day_data in day_datas.to_dict("records") :
    if day_data[dt] != day_data[dt] or day_data["latitude"] != day_data["latitude"] or day_data["longitude"] != day_data["longitude"] :
      continue
    else :
      lats.append(day_data["latitude"])
      lons.append(day_data["longitude"])
      zs.append(day_data[dt])

  data_map = {}

  for day_data in day_datas.to_dict("records") : 
    if day_data[dt] != day_data[dt] or day_data["latitude"] != day_data["latitude"] or day_data["longitude"] != day_data["longitude"] :
        continue
    data_map[Point(day_data["longitude"], day_data["latitude"])] = day_data[dt]

  d = data.copy()
  fig, ax = plt.subplots(figsize=(10,10))
  d.boundary.plot(ax=ax, color='black', alpha = 0.8, linewidth=1, edgecolor='black')
  d["new_gen"] = d.apply(lambda x : func(data_map, x), axis=1)
  d[d["new_gen"].isna() == False]
  plot = d.plot(column="new_gen", vmin=20, vmax=160, ax=ax)
  plt.tick_params(left = False, right = False , labelleft = False , labelbottom = False, bottom = False)
  plot.figure.savefig(f"dataset/images/{dt}.png")
  plt.close()

100%|██████████| 60/60 [00:42<00:00,  1.40it/s]


In [ ]:
with open("dataset/scales.json", "w") as f : 
  json.dump({"min" : min, "max" : max}, f, indent=4)

In [ ]:
!zip -r aqi-images.zip dataset/ --quiet

In [ ]:
!cp aqi-images.zip /gdrive/MyDrive/taim_datasets/